<h1><center>Initial smoke test of the Dell Data Analytics Engine (powered by Starburst)</center></h1>

<a id='the-runtime-environment'></a>
## The runtime environment

This notebook is to allow quick validation that
[Apache Spark](https://spark.apache.org/) code can be run on the
[Dell Data Analytics Engine](https://dell.starburst.io/latest/index.html) -- *powered by [Starburst](httphttps://www.starburst.io/s://)*.

<a id='jupyter-notebook-setup'></a>
## Jupyter notebook setup

<a id='installing-spark'></a>
### Installing Spark

> These instructions where lifted & enhanced from [Colab and PySpark](https://colab.research.google.com/drive/1G894WS7ltIUTusWWmsCnF_zQhQqZCDOc) whose source file can be downloaded from [here](https://github.com/jacobceles/knowledge-repo/blob/master/pyspark/Colab%20and%20PySpark.ipynb) and then used with any Jupyter notebook.

Install Dependencies:

1.   Java 8 (Dell appliance requires 22, but so far 8 is working from the notebook)
2.   Apache Spark with hadoop (Settled on 3.5.1 for starters as needed >= 3.4 for Spark Connect)
3.   Findspark (used to locate the spark in the system)

> If you have issues with spark version, please upgrade to the latest version from [here](https://archive.apache.org/dist/spark/).

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

Set Environment Variables:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [17]:
!ls

sample_data  spark-3.5.1-bin-hadoop3  spark-3.5.1-bin-hadoop3.tgz


Obtain & set Spark Connect uri:

> Full details in the [CLI docs](https://dell.starburst.io/latest/dell-data-processing-engine/cli.html),
but here are the general steps after installation.

Run the following wherever you have the Dell CLI installed.

1. `./dell-data-processing-engine login`
2. `./dell-data-processing-engine submit --spark-connect`
3. Copy the outputted `sparkId` value to your clipboard and replace that with `REPLACE-ME` in next step
4. `./dell-data-processing-engine instance uris REPLACE-ME`
5. Copy the `Spark Connect` uri (starts with `sc://`) to your clipboard and use it in the next code cell


In [14]:
#
# run this cell and past the Spark Connect uri in the textbox that surfaces (and press <enter> OF COURSE; haha)
#

import getpass

sparkConnectUri = input("Spark Connect uri")

Spark Connect urisc://c-fb1e9e1850be42e199231d912b97cccc-grpc.rke2-staging-test-01.eco-tests.eng.starburstdata.net:8787/;token=eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJVd05Rb2RZMEswQVppZXVFNXNtQlk0RU1ZUzlJMnMzcWFibWxXS1FvRGFZIn0.eyJleHAiOjE3NDUzNTgyMDYsImlhdCI6MTc0NTM1NzkwNiwiYXV0aF90aW1lIjoxNzQ1MzU3OTA2LCJqdGkiOiI0ZWU0NjhlMS1iZTM2LTRhMDktOTcxMi0xZmIxYmRiNTg2ZDkiLCJpc3MiOiJodHRwczovL2RlbGwta2V5Y2xvYWsucmtlMi1zdGFnaW5nLXRlc3QtMDEuZWNvLXRlc3RzLmVuZy5zdGFyYnVyc3RkYXRhLm5ldC9yZWFsbXMvZGVsbCIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI1MWU2OTljMS1kZmE0LTRmZDUtOTNhOC1hZmExZThhZDMyNGEiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJ0ZXN0aW5nLWNsaWVudC1pZCIsInNpZCI6ImU5YjJmZGMzLTVjZTMtNDBjYS05YmI2LTg3M2MzMTgyODM3YSIsImFjciI6IjEiLCJhbGxvd2VkLW9yaWdpbnMiOlsiLyoiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwiZGVmYXVsdC1yb2xlcy1kZWxsIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6I

Create the SparkSession:

> Output should look similar to
`<pyspark.sql.connect.session.SparkSession at 0x7fe9f73bbe90>`

In [18]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.remote(sparkConnectUri).getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

/usr/local/lib/python3.11/dist-packages/pyspark/sql/connect/session.py:185: UserWarning: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = ""
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-04-22T21:40:04.660441263+00:00", grpc_status:16, grpc_message:""}"
>
  warnings.warn(str(e))


SparkConnectGrpcException: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = ""
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-04-22T21:40:04.718154059+00:00", grpc_status:16, grpc_message:""}"
>

Create a DataFrame from hard-coded data and display it:

In [19]:
from datetime import datetime, date
from pyspark.sql import Row

df = spark.createDataFrame([
  Row(a=1, b=2, c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
  Row(a=2, b=3, c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
  Row(a=4, b=5, c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)),
  Row(a=8, b=7, c='string4', d=date(2000, 4, 1), e=datetime(2000, 1, 4, 12, 0)),
])
df.show()

SparkConnectGrpcException: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = ""
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-04-22T21:40:11.868776332+00:00", grpc_status:16, grpc_message:""}"
>

<a id='exploring-the-dataset'></a>
## Exploring the Dataset

<a id='loading-the-dataset'></a>
### Loading the Dataset

In [ ]:
# Downloading and preprocessing Cars Data downloaded origianlly from https://perso.telecom-paristech.fr/eagan/class/igr204/datasets
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv

In [ ]:
!ls

In [11]:
from datetime import datetime, date
from pyspark.sql import Row

df = spark.createDataFrame([
  Row(a=1, b=2, c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
  Row(a=2, b=3, c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
  Row(a=4, b=5, c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)),
  Row(a=8, b=7, c='string4', d=date(2000, 4, 1), e=datetime(2000, 1, 4, 12, 0)),
])
df.show()

SparkConnectGrpcException: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAUTHENTICATED
	details = ""
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-04-22T21:37:30.342486511+00:00", grpc_status:16, grpc_message:""}"
>

In [ ]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
distData.show()

In [ ]:
from pyspark import SparkFiles

# CSV file URL
csvUrl = "https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv"

# Add the file to Spark context
spark.sparkContext.addFile(csvUrl)

# Read the CSV file
df = spark.read.csv(SparkFiles.get(csvUrl), header=True, sep=";")

# Show the DataFrame
df.show()


In [ ]:
# Load data from csv to a dataframe.
# header=True means the first row is a header
# sep=';' means the column are seperated using ''
df = spark.read.csv('cars.csv', header=True, sep=";")
df.show(5)

The above command loads our data from into a dataframe (DF). A dataframe is a 2-dimensional labeled data structure with columns of potentially different types.

<a id='viewing-the-dataframe'></a>
### Viewing the Dataframe

There are a couple of ways to view your dataframe(DF) in PySpark:

1.   `df.take(5)` will return a list of five Row objects.
2.   `df.collect()` will get all of the data from the entire DataFrame. Be really careful when using it, because if you have a large data set, you can easily crash the driver node.
3.   `df.show()` is the most commonly used method to view a dataframe. There are a few parameters we can pass to this method, like the number of rows and truncaiton. For example, `df.show(5, False)` or ` df.show(5, truncate=False)` will show the entire data wihtout any truncation.
4.   `df.limit(5)` will **return a new DataFrame** by taking the first n rows. As spark is distributed in nature, there is no guarantee that `df.limit()` will give you the same results each time.

Let us see some of them in action below:

In [ ]:
df.show(5, truncate=False)

In [ ]:
df.limit(5)

<a id='viewing-dataframe-columns'></a>
### Viewing Dataframe Columns

In [ ]:
df.columns

<a id='dataframe-schema'></a>
### Dataframe Schema

There are two methods commonly used to view the data types of a dataframe:

In [ ]:
df.dtypes

In [ ]:
df.printSchema()

<a id='implicit-schema-inference'></a>
#### Inferring Schema Implicitly

We can use the parameter `inferschema=true` to infer the input schema automatically while loading the data. An example is shown below:

In [ ]:
df = spark.read.csv('cars.csv', header=True, sep=";", inferSchema=True)
df.printSchema()

As you can see, the datatype has been infered automatically spark with even the correct precison for decimal type. A problem that might arise here is that sometimes, when you have to read multiple files with different schemas in different files, there might be an issue with implicit inferring leading to null values in some columns. Therefore, let us also see how to define schemas explicitly.

<a id='explicit-schema-inference'></a>
#### Defining Schema Explicitly

In [ ]:
from pyspark.sql.types import *
df.columns

In [ ]:
# Creating a list of the schema in the format column_name, data_type
labels = [
     ('Car',StringType()),
     ('MPG',DoubleType()),
     ('Cylinders',IntegerType()),
     ('Displacement',DoubleType()),
     ('Horsepower',DoubleType()),
     ('Weight',DoubleType()),
     ('Acceleration',DoubleType()),
     ('Model',IntegerType()),
     ('Origin',StringType())
]

In [ ]:
# Creating the schema that will be passed when reading the csv
schema = StructType([StructField (x[0], x[1], True) for x in labels])
schema

In [ ]:
df = spark.read.csv('cars.csv', header=True, sep=";", schema=schema)
df.printSchema()
# The schema comes as we gave!

In [ ]:
df.show(truncate=False)

As we can see here, the data has been successully loaded with the specified datatypes.

<a id='dataframe-operations-on-columns'></a>
## DataFrame Operations on Columns

We will go over the following in this section:

1.   Selecting Columns
2.   Selecting Multiple Columns
3.   Adding New Columns
4.   Renaming Columns
5.   Grouping By Columns
6.   Removing Columns



<a id='selecting-columns'></a>
### Selecting Columns

There are multiple ways to do a select in PySpark. You can find how they differ and how each below:

In [ ]:
# 1st method
# Column name is case sensitive in this usage
print(df.Car)
print("*"*20)
df.select(df.Car).show(truncate=False)

**NOTE:**

> **We can't always use the dot notation because this will break when the column names have reserved names or attributes to the data frame class. Additionally, the column names are case sensitive in nature so we need to always make sure the column names have been changed to a paticular case before using it.**



In [ ]:
# 2nd method
# Column name is case insensitive here
print(df['car'])
print("*"*20)
df.select(df['car']).show(truncate=False)

In [ ]:
# 3rd method
# Column name is case insensitive here
from pyspark.sql.functions import col
df.select(col('car')).show(truncate=False)

<a id='selecting-multiple-columns'></a>
### Selecting Multiple Columns

In [ ]:
# 1st method
# Column name is case sensitive in this usage
print(df.Car, df.Cylinders)
print("*"*40)
df.select(df.Car, df.Cylinders).show(truncate=False)

In [ ]:
# 2nd method
# Column name is case insensitive in this usage
print(df['car'],df['cylinders'])
print("*"*40)
df.select(df['car'],df['cylinders']).show(truncate=False)

In [ ]:
# 3rd method
# Column name is case insensitive in this usage
from pyspark.sql.functions import col
df.select(col('car'),col('cylinders')).show(truncate=False)

<a id='adding-new-columns'></a>
### Adding New Columns

We will take a look at three cases here:

1.   Adding a new column
2.   Adding multiple columns
3.   Deriving a new column from an exisitng one

In [ ]:
# CASE 1: Adding a new column
# We will add a new column called 'first_column' at the end
from pyspark.sql.functions import lit
df = df.withColumn('first_column',lit(1))
# lit means literal. It populates the row with the literal value given.
# When adding static data / constant values, it is a good practice to use it.
df.show(5,truncate=False)

In [ ]:
# CASE 2: Adding multiple columns
# We will add two new columns called 'second_column' and 'third_column' at the end
df = df.withColumn('second_column', lit(2)) \
       .withColumn('third_column', lit('Third Column'))
# lit means literal. It populates the row with the literal value given.
# When adding static data / constant values, it is a good practice to use it.
df.show(5,truncate=False)

In [ ]:
# CASE 3: Deriving a new column from an exisitng one
# We will add a new column called 'car_model' which has the value of car and model appended together with a space in between
from pyspark.sql.functions import concat
df = df.withColumn('car_model', concat(col("Car"), lit(" "), col("model")))
# lit means literal. It populates the row with the literal value given.
# When adding static data / constant values, it is a good practice to use it.
df.show(5,truncate=False)

As we can see, the new column car model has been created from existing columns. Since our aim was to create a column which has the value of car and model appended together with a space in between we have used the `concat` operator.

<a id='renaming-columns'></a>
### Renaming Columns

We use the `withColumnRenamed` function to rename a columm in PySpark. Let us see it in action below:

In [ ]:
#Renaming a column in PySpark
df = df.withColumnRenamed('first_column', 'new_column_one') \
       .withColumnRenamed('second_column', 'new_column_two') \
       .withColumnRenamed('third_column', 'new_column_three')
df.show(truncate=False)

<a id='grouping-by-columns'></a>
### Grouping By Columns

Here, we see the Dataframe API way of grouping values. We will discuss how to:


1.   Group By a single column
2.   Group By multiple columns

In [ ]:
# Group By a column in PySpark
df.groupBy('Origin').count().show(5)

In [ ]:
# Group By multiple columns in PySpark
df.groupBy('Origin', 'Model').count().show(5)

<a id='removing-columns'></a>
### Removing Columns

In [ ]:
#Remove columns in PySpark
df = df.drop('new_column_one')
df.show(5,truncate=False)

In [ ]:
#Remove multiple columnss in one go
df = df.drop('new_column_two') \
       .drop('new_column_three')
df.show(5,truncate=False)

<a id='dataframe-operations-on-rows'></a>
## DataFrame Operations on Rows

We will discuss the follwoing in this section:

1.   Filtering Rows
2. 	 Get Distinct Rows
3.   Sorting Rows
4.   Union Dataframes



<a id='filtering-rows'></a>
### Filtering Rows

In [ ]:
# Filtering rows in PySpark
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
europe_filtered_count = df.filter(col('Origin')=='Europe').count()
print("EUROPE FILTERED RECORD COUNT: " + str(europe_filtered_count))
df.filter(col('Origin')=='Europe').show(truncate=False)

In [ ]:
# Filtering rows in PySpark based on Multiple conditions
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
europe_filtered_count = df.filter((col('Origin')=='Europe') &
                                  (col('Cylinders')==4)).count() # Two conditions added here
print("EUROPE FILTERED RECORD COUNT: " + str(europe_filtered_count))
df.filter(col('Origin')=='Europe').show(truncate=False)

<a id='get-distinct-rows'></a>
### Get Distinct Rows

In [ ]:
#Get Unique Rows in PySpark
df.select('Origin').distinct().show()

In [ ]:
#Get Unique Rows in PySpark based on mutliple columns
df.select('Origin','model').distinct().show()

<a id='sorting-rows'></a>
### Sorting Rows

In [ ]:
# Sort Rows in PySpark
# By default the data will be sorted in ascending order
df.orderBy('Cylinders').show(truncate=False)

In [ ]:
# To change the sorting order, you can use the ascending parameter
df.orderBy('Cylinders', ascending=False).show(truncate=False)

In [ ]:
# Using groupBy aand orderBy together
df.groupBy("Origin").count().orderBy('count', ascending=False).show(10)

<a id='union-dataframes'></a>
### Union Dataframes

You will see three main methods for performing union of dataframes. It is important to know the difference between them and which one is preferred:

*   `union()` – It is used to merge two DataFrames of the same structure/schema. If schemas are not the same, it returns an error
*   `unionAll()` – This function is deprecated since Spark 2.0.0, and replaced with union()
*   `unionByName()` - This function is used to merge two dataframes based on column name.

> Since `unionAll()` is deprecated, **`union()` is the preferred method for merging dataframes.**
<br>
> The difference between `unionByName()` and `union()` is that `unionByName()` resolves columns by name, not by position.

In other SQLs, Union eliminates the duplicates but UnionAll merges two datasets, thereby including duplicate records. But, in PySpark, both behave the same and includes duplicate records. The recommendation is to use `distinct()` or `dropDuplicates()` to remove duplicate records.

In [ ]:
# CASE 1: Union When columns are in order
df = spark.read.csv('cars.csv', header=True, sep=";", inferSchema=True)
europe_cars = df.filter((col('Origin')=='Europe') & (col('Cylinders')==5))
japan_cars = df.filter((col('Origin')=='Japan') & (col('Cylinders')==3))
print("EUROPE CARS: "+str(europe_cars.count()))
print("JAPAN CARS: "+str(japan_cars.count()))
print("AFTER UNION: "+str(europe_cars.union(japan_cars).count()))

**Result:**

> As you can see here, there were 3 cars from Europe with 5 Cylinders, and 4 cars from Japan with 3 Cylinders. After union, there are 7 cars in total.



In [ ]:
# CASE 1: Union When columns are not in order
# Creating two dataframes with jumbled columns
df1 = spark.createDataFrame([[1, 2, 3]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[4, 5, 6]], ["col1", "col2", "col0"])
df1.unionByName(df2).show()

**Result:**

> As you can see here, the two dataframes have been successfully merged based on their column names.



<a id='common-data-manipulation-functions'></a>
## Common Data Manipulation Functions

In [ ]:
# Functions available in PySpark
from pyspark.sql import functions
# Similar to python, we can use the dir function to view the avaiable functions
print(dir(functions))

<a id='string-functions'></a>
### String Functions

In [ ]:
# Loading the data
from pyspark.sql.functions import col
df = spark.read.csv('cars.csv', header=True, sep=";", inferSchema=True)

**Display the Car column in exisitng, lower and upper characters, and the first 4 characters of the column**

In [ ]:
from pyspark.sql.functions import col,lower, upper, substring
# Prints out the details of a function
help(substring)
# alias is used to rename the column name in the output
df.select(col('Car'),lower(col('Car')),upper(col('Car')),substring(col('Car'),1,4).alias("concatenated value")).show(5, False)

**Concatenate the Car column and Model column and add a space between them.**

In [ ]:
from pyspark.sql.functions import concat
df.select(col("Car"),col("model"),concat(col("Car"), lit(" "), col("model"))).show(5, False)

<a id='numeric-functions'></a>
### Numeric functions

**Show the oldest date and the most recent date**

In [ ]:
from pyspark.sql.functions import min, max
df.select(min(col('Weight')), max(col('Weight'))).show()

**Add 10 to the minimum and maximum weight**

In [ ]:
from pyspark.sql.functions import min, max, lit
df.select(min(col('Weight'))+lit(10), max(col('Weight')+lit(10))).show()

<a id='operations-on-date'></a>
### Operations on Date

> [PySpark follows SimpleDateFormat table of Java. Click here to view the docs.](https://docs.oracle.com/javase/7/docs/api/java/text/SimpleDateFormat.html)

In [ ]:
from pyspark.sql.functions import to_date, to_timestamp, lit
df = spark.createDataFrame([('2019-12-25 13:30:00',)], ['DOB'])
df.show()
df.printSchema()

In [ ]:
df = spark.createDataFrame([('2019-12-25 13:30:00',)], ['DOB'])
df = df.select(to_date(col('DOB'),'yyyy-MM-dd HH:mm:ss'), to_timestamp(col('DOB'),'yyyy-MM-dd HH:mm:ss'))
df.show()
df.printSchema()

In [ ]:
df = spark.createDataFrame([('25/Dec/2019 13:30:00',)], ['DOB'])
df = df.select(to_date(col('DOB'),'dd/MMM/yyyy HH:mm:ss'), to_timestamp(col('DOB'),'dd/MMM/yyyy HH:mm:ss'))
df.show()
df.printSchema()

**What is 3 days earlier that the oldest date and 3 days later than the most recent date?**

In [ ]:
from pyspark.sql.functions import date_add, date_sub
# create a dummy dataframe
df = spark.createDataFrame([('1990-01-01',),('1995-01-03',),('2021-03-30',)], ['Date'])
# find out the required dates
df.select(date_add(max(col('Date')),3), date_sub(min(col('Date')),3)).show()

<a id='joins-in-pyspark'></a>
## Joins in PySpark

In [ ]:
# Create two dataframes
cars_df = spark.createDataFrame([[1, 'Car A'],[2, 'Car B'],[3, 'Car C']], ["id", "car_name"])
car_price_df = spark.createDataFrame([[1, 1000],[2, 2000],[3, 3000]], ["id", "car_price"])
cars_df.show()
car_price_df.show()

In [ ]:
# Executing an inner join so we can see the id, name and price of each car in one row
cars_df.join(car_price_df, cars_df.id == car_price_df.id, 'inner').select(cars_df['id'],cars_df['car_name'],car_price_df['car_price']).show(truncate=False)

As you can see, we have done an inner join between two dataframes. The following joins are supported by PySpark:
1. inner (default)
2. cross
3. outer
4. full
5. full_outer
6. left
7. left_outer
8. right
9. right_outer
10. left_semi
11. left_anti

<a id='spark-sql'></a>
## Spark SQL

SQL has been around since the 1970s, and so one can imagine the number of people who made it their bread and butter. As big data came into popularity, the number of professionals with the technical knowledge to deal with it was in shortage. This led to the creation of Spark SQL. To quote the docs:<br>
>Spark SQL is a Spark module for structured data processing. Unlike the basic Spark RDD API, the interfaces provided by Spark SQL provide Spark with more information about the structure of both the data and the computation being performed. Internally, Spark SQL uses this extra information to perform extra optimizations.

Basically, what you need to know is that Spark SQL is used to execute SQL queries on big data. Spark SQL can also be used to read data from Hive tables and views. Let me explain Spark SQL with an example.


In [ ]:
# Load data
df = spark.read.csv('cars.csv', header=True, sep=";")
# Register Temporary Table
df.createOrReplaceTempView("temp")
# Select all data from temp table
spark.sql("select * from temp limit 5").show()
# Select count of data in table
spark.sql("select count(*) as total_count from temp").show()

As you can see, we registered the dataframe as temporary table and then ran basic SQL queries on it. How amazing is that?!<br>
If you are a person who is more comfortable with SQL, then this feature is truly a blessing for you! But this raises a question:
> *Should I just keep using Spark SQL all the time?*

And the answer is, _**it depends**_.<br>
So basically, the different functions acts in differnet ways, and depending upon the type of action you are trying to do, the speed at which it completes execution also differs. But as time progress, this feature is getting better and better, so hopefully the difference should be a small margin. There are plenty of analysis done on this, but nothing has a definite answer yet. You can read this [comparative study done by horton works](https://community.cloudera.com/t5/Community-Articles/Spark-RDDs-vs-DataFrames-vs-SparkSQL/ta-p/246547) or the answer to this [stackoverflow question](https://stackoverflow.com/questions/45430816/writing-sql-vs-using-dataframe-apis-in-spark-sql) if you are still curious about it.

<a id='rdd'></a>
## RDD

> With map, you define a function and then apply it record by record. Flatmap returns a new RDD by first applying a function to all of the elements in RDDs and then flattening the result. Filter, returns a new RDD. Meaning only the elements that satisfy a condition. With reduce, we are taking neighboring elements and producing a single combined result.
For example, let's say you have a set of numbers. You can reduce this to its sum by providing a function that takes as input two values and reduces them to one.

Some of the reasons you would use a dataframe over RDD are:
1.   It's ability to represnt data as rows and columns. But this also means it can only hold structred and semi-structured data.
2.   It allows processing data in different formats (AVRO, CSV, JSON, and storage system HDFS, HIVE tables, MySQL).
3. It's superior job Optimization capability.
4. DataFrame API is very easy to use.





In [ ]:
cars = spark.sparkContext.textFile('cars.csv')
print(cars.first())
cars_header = cars.first()
cars_rest = cars.filter(lambda line: line!=cars_header)
print(cars_rest.first())

**How many cars are there in our csv data?**

In [ ]:
cars_rest.map(lambda line: line.split(";")).count()

**Display the Car name, MPG, Cylinders, Weight and Origin for the cars Originating in Europe**

In [ ]:
# Car name is column  0
(cars_rest.filter(lambda line: line.split(";")[8]=='Europe').
 map(lambda line: (line.split(";")[0],
    line.split(";")[1],
    line.split(";")[2],
    line.split(";")[5],
    line.split(";")[8])).collect())

**Display the Car name, MPG, Cylinders, Weight and Origin for the cars Originating in either Europe or Japan**

In [ ]:
# Car name is column  0
(cars_rest.filter(lambda line: line.split(";")[8] in ['Europe','Japan']).
 map(lambda line: (line.split(";")[0],
    line.split(";")[1],
    line.split(";")[2],
    line.split(";")[5],
    line.split(";")[8])).collect())

<a id='user-defined-functions-udf'></a>
## User-Defined Functions (UDF)

PySpark User-Defined Functions (UDFs) help you convert your python code into a scalable version of itself. It comes in handy more than you can imagine, but beware, as the performance is less when you compare it with pyspark functions. You can view examples of how UDF works [here](https://docs.databricks.com/spark/latest/spark-sql/udf-python.html). What I will give in this section is some theory on how it works, and why it is slower.

When you try to run a UDF in PySpark, each executor creates a python process. Data will be serialised and deserialised between each executor and python. This leads to lots of performance impact and overhead on spark jobs, making it less efficent than using spark dataframes. Apart from this, sometimes you might have memory issues while using UDFs. The Python worker consumes huge off-heap memory and so it often leads to memoryOverhead, thereby failing your job. Keeping these in mind, I wouldn't recommend using them, but at the end of the day, your choice.

<a id='common-questions'></a>
# Common Questions

<a id='recommended-ide'></a>
## Recommended IDE

I personally prefer [PyCharm](https://www.jetbrains.com/pycharm/) while coding in Python/PySpark. It's based on IntelliJ IDEA so it has a lot of features! And the main advantage I have felt is the ease of installing PySpark and other packages. You can customize it with themes and plugins, and it lets you enhance productivity while coding by providing some features like suggestions, local VCS etc.

<a id='submitting-a-spark-job'></a>
## Submitting a Spark Job

The python syntax for running jobs is: `python <file_name>.py <arg1> <arg2> ...`
<br>But when you submit a spark job you have to use spark-submit to run the application.

Here is a simple example of a spark-submit command:
`spark-submit filename.py --named_argument 'arguemnt value'`<br>
Here, named_argument is an argument that you are reading from inside your script.

There are other options you can pass in the command, like:<br>
`--py-files` which helps you pass a python file to read in your file,<br>
`--files` which helps pass other files like txt or config,<br>
`--deploy-mode` which tells wether to deploy your worker node on cluster or locally <br>
`--conf` which helps pass different configurations, like memoryOverhead, dynamicAllocation etc.

There is an [entire page](https://spark.apache.org/docs/latest/submitting-applications.html) in spark documentation dedicated to this. I highly recommend you go through it once.

<a id='creating-dataframes'></a>
## Creating Dataframes

When getting started with dataframes, the most common question is: *'How do I create a dataframe?'* <br>
Below, you can see how to create three kinds of dataframes:

### Create a totally empty dataframe

In [ ]:
from pyspark.sql.types import StructType
sc = spark.sparkContext
#Create empty df
schema = StructType([])
empty = spark.createDataFrame(sc.emptyRDD(), schema)
empty.show()

### Create an empty dataframe with header

In [ ]:
from pyspark.sql.types import StructType, StructField
#Create empty df with header
schema_header = StructType([StructField("name", StringType(), True)])
empty_with_header = spark.createDataFrame(sc.emptyRDD(), schema_header)
empty_with_header.show()

### Create a dataframe with header and data

In [ ]:
from pyspark.sql import Row
mylist = [
  {"name":'Alice',"age":13},
  {"name":'Jacob',"age":24},
  {"name":'Betty',"age":135},
]
spark.createDataFrame(Row(**x) for x in mylist).show()

In [ ]:
# You can achieve the same using this - note that we are using spark context here, not a spark session
from pyspark.sql import Row
df = sc.parallelize([
        Row(name='Alice', age=13),
        Row(name='Jacob', age=24),
        Row(name='Betty', age=135)]).toDF()
df.show()

<a id='drop-duplicates'></a>
## Drop Duplicates

As mentioned earlier, there are two easy to remove duplicates from a dataframe. We have already seen the usage of distinct under <a href="#get-distinct-rows">Get Distinct Rows</a>  section.
I will expalin how to use the `dropDuplicates()` function to achieve the same.

> `drop_duplicates()` is an alias for `dropDuplicates()`

In [ ]:
from pyspark.sql import Row
from pyspark.sql import Row
mylist = [
  {"name":'Alice',"age":5,"height":80},
  {"name":'Jacob',"age":24,"height":80},
  {"name":'Alice',"age":5,"height":80}
]
df = spark.createDataFrame(Row(**x) for x in mylist)
df.dropDuplicates().show()

`dropDuplicates()` can also take in an optional parameter called *subset* which helps specify the columns on which the duplicate check needs to be done on.

In [ ]:
df.dropDuplicates(subset=['height']).show()

<a id='fine-tuning-a-pyspark-job'></a>
## Fine Tuning a Spark Job

Before we begin, please note that this entire section is written purely based on experience. It might differ with use cases, but it will help you get a better understanding of what you should be looking for, or act as a guidance to achieve your aim.

>Spark Performance Tuning refers to the process of adjusting settings to record for memory, cores, and instances used by the system. This process guarantees that the Spark has a flawless performance and also prevents bottlenecking of resources in Spark.

Considering you are using Amazon EMR to execute your spark jobs, there are three aspects you need to take care of:
1. EMR Sizing
2. Spark Configurations
3. Job Tuning



<a id='emr-sizing'></a>
### EMR Sizing

Sizing your EMR is extremely important, as this affects the efficency of your spark jobs. Apart from the cost factor, the maximum number of nodes and memory your job can use will be decided by this. If you spin up a EMR with high specifications, that obviously means you are paying more for it, so we should ideally utilize it to the max. These are the guidelines that I follow to make sure the EMR is rightly sized:

1. Size of the input data (include all the input data) on the disk.
2. Whether the jobs have transformations or just a straight pass through.<br> Assess the joins and the complex joins involved.
3. Size of the output data on the disk.

Look at the above criteria against the memory you need to process, and the disk space you would need. Start with a small configuration, and keep adding nodes to arrive at an optimal configuration. In case you are wondering about the *Execution time vs EMR configuration* factor, please understand that it is okay for a job to run longer, rather than adding more resources to the cluster. For example, it is okay to run a job for 40 mins job on a 5 node cluster, rather than running a job in 10 mins on a 15 node cluster.


Another thing you need to know about EMRs, are the different kinds of EC2 instance types provided by Amazon. I will briefly talk about them, but I strongly recommend you to read more about it from the [official documentation](https://aws.amazon.com/ec2/instance-types/). There are 5 types of instance classes. Based on the job you want to run, you can decide which one to use:

>Instance Class | Description
>--- | ---
>General purpose | Balance of compute, memory and networking resources
>Compute optimized | Ideal for compute bound applications that benefit from high performance processors
>Memory optimized | Designed to deliver fast performance for workloads that process large data sets in memory
>Storage optimized | For workloads that require high, sequential read and write access to very large data sets on local storage
>GPU instances | Use hardware accelerators, or co-processors, to perform high demanding functions, more efficiently than is possible in software running on CPUs

The configuration (memory, storage, cpu, network performance) will differ based on the instance class you choose.<br>
To help make life easier, here is what I do when I get into a predicament about which one to go with: <br>
 1. Visit [ec2instances](https://www.ec2instances.info/)
 2. Choose the EC2 instances in question
 3. Click on compare selected

This will easily help you undesrstand what you are getting into, and thereby help you make the best choice! The site was built by [Garret Heaton](https://github.com/powdahound)(founder of Swoot), and has helped me countless number of times to make an informed decision.

<a id='spark-configurations'></a>
### Spark Configurations

There are a ton of [configurations](https://spark.apache.org/docs/latest/configuration.html) that you can tweak when it comes to Spark. Here, I will be noting down some of the configurations which I use, which have worked well for me. Alright! let's get into it!

#### Job Scheduling

When you submit your job in a cluster, it will be given to Spark Schedulers, which is responsible for materializing a logical plan for your job. There are two types of [job scheduling](https://spark.apache.org/docs/latest/job-scheduling.html):
1. FIFO<br>
By default, Spark’s scheduler runs jobs in FIFO fashion. Each job is divided into stages (e.g. map and reduce phases), and the first job gets priority on all available resources while its stages have tasks to launch, then the second job gets priority, etc. If the jobs at the head of the queue don’t need to use the whole cluster, later jobs can start to run right away, but if the jobs at the head of the queue are large, then later jobs may be delayed significantly.
2. FAIR<br>
The fair scheduler supports grouping jobs into pools and setting different scheduling options (e.g. weight) for each pool. This can be useful to create a high-priority pool for more important jobs, for example, or to group the jobs of each user together and give users equal shares regardless of how many concurrent jobs they have instead of giving jobs equal shares. This approach is modeled after the Hadoop Fair Scheduler.

> I personally prefer using the FAIR mode, and this can be set by adding `.config("spark.scheduler.mode", "FAIR")` when you create your SparkSession.


#### Serializer

We have two types of [serializers](https://spark.apache.org/docs/latest/tuning.html#data-serialization) available:
1. Java serialization
2. Kryo serialization

Kryo is significantly faster and more compact than Java serialization (often as much as 10x), but does not support all Serializable types and requires you to register the classes you’ll use in the program in advance for best performance.

Java serialization is used by default because if you have custom class that extends Serializable it can be easily used. You can also control the performance of your serialization more closely by extending java.io.Externalizable

> The general recommendation is to use Kyro as the serializer whenver possible, as it leads to much smaller sizes than Java serialization. It can be added by using `.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")` when you create your SparkSession.


#### Shuffle Behaviour

It is generally a good idea to compress the output file after the map phase. The `spark.shuffle.compress` property decides whether to do the compression or not. The compression used is `spark.io.compression.codec`.

> The property can be added by using `.config("spark.shuffle.compress", "true")` when you create your SparkSession.

#### Compression and Serialization

There are 4 defaiult codecs spark provides to compress internal data such as RDD partitions, event log, broadcast variables and shuffle outputs. They are:

1. lz4
2. lzf
3. snappy
4. zstd

> The decision on which to use rests upon the use case. I generally use the `snappy` compression. Google created Snappy because they needed something that offered very fast compression at the expense of final size. Snappy is fast, stable and free, but it increases the size more than the other codecs. At the same time, since compute costs will be less, it seems like balanced trade off. The property can be added by using `.config("spark.io.compression.codec", "snappy")` when you create your SparkSession.

This [session](https://databricks.com/session/best-practice-of-compression-decompression-codes-in-apache-spark) explains the best practice of compression/decompression codes in Apache Spark. I recommend you to take a look at it before taking a decision.

#### Scheduling

The property `spark.speculation` performs speculative execution of tasks. This means if one or more tasks are running slowly in a stage, they will be re-launched. Speculative execution will not stop the slow running task but it launches the new task in parallel.

> I usually disable this option by adding `.config("spark.speculation", "false") ` when I create the SparkSession.

#### Application Properties

There are mainly two application properties that you should know about:

1. spark.driver.memoryOverhead - The amount of off-heap memory to be allocated per driver in cluster mode, in MiB unless otherwise specified. This is memory that accounts for things like VM overheads, interned strings, other native overheads, etc. This tends to grow with the container size (typically 6-10%). This option is currently supported on YARN and Kubernetes.

2. spark.executor.memoryOverhead - The amount of off-heap memory to be allocated per executor, in MiB unless otherwise specified. This is memory that accounts for things like VM overheads, interned strings, other native overheads, etc. This tends to grow with the executor size (typically 6-10%). This option is currently supported on YARN and Kubernetes.

> If you ever face an issue like `Container killed by YARN for exceeding memory limits`, know that it is because you have not specified enough memory Overhead for your job to successfully execute. The default value for Overhead is 10% of avaialbe memory (driver/executor sepearte), with minimum of 384.



#### Dynamic Allocation

Lastly, I want to talk about Dynamic Allocation. This is a feature I constantly use while executing my jobs. This property is by defualt set to False. As the name suggests, it sets whether to use dynamic resource allocation, which scales the number of executors registered with this application up and down based on the workload. Truly a wonderful feature, and the greatest benefit of using it is that it will help make the best use of all the resources you have! The disadvantage of this feature is that it does not shine well when you have to execute tasks in parallel. Since most of the resources will be used by the first task, the second one will have to wait till some resource gets released. At the same time, if both get submitted at the exact same time, the resources will be shared between them, although not equally. Also, it is not guaranteed to *always* use the most optimal configurations. But in all my tests, the results have been great!

> If you are planning on using this feature, you can pass the configurations as required through the spark-submit command. The four configurations which you will have to keep in mind are:<br>
```
--conf spark.dynamicAllocation.enabled=true
--conf spark.dynamicAllocation.initialExecutors
--conf spark.dynamicAllocation.minExecutors
--conf spark.dynamicAllocation.maxExecutors
```

You can read more about this feature [here](https://spark.apache.org/docs/latest/configuration.html#dynamic-allocation) and [here](https://stackoverflow.com/questions/40200389/how-to-execute-spark-programs-with-dynamic-resource-allocation).





<a id='job-tuning'></a>
### Job Tuning

Apart from EMR and Spark tuning, there is another way to approach opttimizations, and that is by tuning your job itself to produce results efficently. I will be going over some such techniques which will help you achieve this. The [Spark Programming Guide](https://spark.apache.org/docs/2.1.1/programming-guide.html) talks more about these concepts in detail. If you guys prefer watching a video over reading, I highly recommend [A Deep Dive into Proper Optimization for Spark Jobs](https://youtu.be/daXEp4HmS-E) by Daniel Tomes from Databricks, which I found really useful and informative!

#### Broadcast Joins (Broadcast Hash Join)

For some jobs, the efficenecy can be increased by caching them in memory. Broadcast Hash Join(BHJ) is such a technique which will help you optimize join queries when the size of one side of the data is low.
>BroadCast joins are the fastest but the drawaback is that it will consume more memory on both the executor and driver.

This following steps give a sneak peek into how it works, which will help you understand the use cases where it can be used:<br>
1. Input file(smaller of the two tables) to be broadcasted is read by the executors in parallel into its working memory.
2. All the data from the executors is collected into driver (Hence, the need for higher memory at driver).
3. The driver then broadcasts the combined dataset (full copy) into each executor.
4. The size of the broadcasted dataset could be several (10-20+) times bigger the input in memory due to factors like deserialization.
5. Executors will end up storing the parts it read first, and also the full copy, thereby leading to a high memory requirement.

Some things to keep in mind about BHJ:
1. It is advisable to use broadcast joins on small datasets only (dimesnion table, for example).
2. Spark does not guarantee BHJ is always chosen, since not all cases (e.g. full outer join) support BHJ.
3. You could notice skews in tasks due to uneven partition sizes; especially during aggregations, joins etc. This can be evened out by introducing Salt value (random value).<br>*Suggested formula for salt value:* random(0 – (shuffle partition count – 1))


#### Spark Partitions

A partition in spark is an atomic chunk of data (logical division of data) stored on a node in the cluster. Partitions are the basic units of parallelism in Spark. Having too large a number of partitions or too few is not an ideal solution. The number of partitions in spark should be decided based on the cluster configuration and requirements of the application. Increasing the number of partitions will make each partition have less data or no data at all. Generally, spark partitioning can be broken down in three ways:
1. Input
2. Shuffle
3. Output


##### Input

Spark usually does a good job of figuring the ideal configuration for this one, except in very particular cases. It is advisable to use the spark default unless:
1. Increase parallelism
2. Heavily nested data
3. Generating data (explode)
4. Source is not optimal
5. You are using UDFs

`spark.sql.files.maxpartitionBytes`: This property indicates the maximum number of bytes to pack into a single partition when reading files (Default 128 MB) . Use this to increase the parallelism in reading input data. For example, if you have more cores, then you can increase the number of parallel tasks which will ensure usage of the all the cores of the cluster, and increase the speed of the task.

##### Shuffle

One of the major reason why most jobs lags in performance is, for the majority of the time, because they get the shuffle partitions count wrong. By default, the value is set to 200. In almost all situations, this is not ideal. If you are dealing with shuffle satge of less than 20 GB, 200 is fine, but otherwise this needs to be changed. For most cases, you can use the following equation to find the right value:
>`Partition Count = Stage Input Data / Target Size` where <br>
`Largest Shuffle Stage (Target Size) < 200MB/partition` in most cases.<br>
`spark.sql.shuffle.partitions` property is used to set the ideal partition count value.

If you ever notice that target size at the range of TBs, there is something terribly wrong, and you might want to change it back to 200, or recalculate it. Shuffle partitions can be configured for every action (not transformation) in the spark script.

Let us use an example to explain this scenario: <br>
Assume shuffle stage input = 210 GB. <br>
Partition Count = Stage Input Data / Target Size = 210000 MB/200 MB = 1050. <br>
As you can see, my shuffle partitions should be 1050, not 200.

But, if your cluster has 2000 cores, then set your shuffle partitions to 2000.
>In a large cluster dealing with a large data job, never set your shuffle partitions less than your total core count.



Shuffle stages almost always precede the write stages and having high shuffle partition count creates small files in the output. To address this, use localCheckPoint just before write & do a coalesce call. This localCheckPoint writes the Shuffle Partition to executor local disk and then coalesces into lower partition count and hence improves the overall performance of both shuffle stage and write stage.

##### Output

There are different methods to write the data. You can control the size, composition, number of files in the output and even the number of records in each file while writing the data. While writing the data, you can increase the parallelism, thereby ensuring you use all the resources that you have. But this approach would lead to a larger number of smaller files. Usually, this isn't a problem, but if you want bigger files, you will have to use one of the compaction techniques, preferably in a cluster with lesser configuration. There are multiple ways to change the composition of the output. Keep these two in mind about composition:
1. Coalesce: Use this to reduce the number of partitions.
2. Repartition: Use this very rarely, and never to reduce the number of partitions<br>
    a. Range Paritioner - It partitions the data either based on some sorted order OR set of sorted ranges of keys. <br>
    b. Hash Partioner - It spreads around the data in the partitioning based upon the key value. Hash partitioning can make distributed data skewed.

<a id='best-practices'></a>
### Best Practices

Try to incorporate these to your coding habits for better performance:
1.   Do not use NOT IN use NOT EXISTS.
2.   Remove Counts, Distinct Counts (use approxCountDIstinct).
3.   Drop Duplicates early.
4.   Always prefer SQL functions over PandasUDF.
5.   Use Hive partitions effectively.
6.   Leverage Spark UI effectively.
7.   Avoid Shuffle Spills.
8.   Aim for target cluster utilization of atleast 70%.



In [ ]:
%%shell
jupyter nbconvert --to html /content/Colab_and_PySpark.ipynb